<a href="https://colab.research.google.com/github/Server-security-visualization/AI/blob/main/KISA/Malware/Training_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Setting dataset and github

In [ ]:
!mkdir -p ./dataset/KISA
!unzip /content/drive/MyDrive/server_security/colab/KISA/Malware/dataset/img256.zip -d ./dataset/KISA
!unzip /content/drive/MyDrive/server_security/colab/MS\ Malware/img256.zip -d ./dataset/MS/img256
!cp /content/drive/MyDrive/server_security/colab/KISA/Malware/dataset/malware1.csv ./dataset/KISA
!git clone https://github.com/Server-security-visualization/AI.git

# import

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
import datetime
import shutil
from matplotlib import pyplot as plt
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Hyper Parameter

In [ ]:
image_w = 256
image_h = 256
input_shape = (image_w, image_h, 1)

# Load

# KISA All Load
train, val 데이터셋

In [ ]:
kisa_img_path = "./dataset/KISA/img256"

In [ ]:
malware_df =pd.read_csv("./dataset/KISA/malware1.csv")
malware_df.head()

In [ ]:
x = []
y = []

for idx, row in tqdm(malware_df.iterrows()):
    path = f"{kisa_img_path}/{row['png']}"
    malware_class = row['malware']
    
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = img.reshape(image_w,image_h,1)
    
    x.append(img/255.)
    y.append(malware_class)
    
x = np.array(x)
y = np.array(y)
    
train_x, val_x, train_y, val_y = train_test_split(x, 
                                                  y, 
                                                  stratify=y, 
                                                  test_size=0.1)

del x
del y

In [ ]:
train_x.shape, val_x.shape, train_y.shape, val_y.shape

## KISA Load
train용 데이터셋

In [ ]:
kisa_img_path = "./dataset/KISA/img256"

In [ ]:
malware_df =pd.read_csv("./dataset/KISA/malware1.csv")
malware_df.head()

In [ ]:
train_x = []
train_y = []

for idx, row in tqdm(malware_df.iterrows()):
    path = f"{kisa_img_path}/{row['png']}"
    malware_class = row['malware']
    
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = img.reshape(image_w,image_h,1)
    
    train_x.append(img/255.)
    train_y.append(malware_class)
    
train_x = np.array(train_x)
train_y = np.array(train_y)

## MS Load
validation 데이터셋

In [ ]:
ms_img_path = "./dataset/MS/img256"

In [ ]:
val_x = []
val_y = []

for path in tqdm(glob(f"{ms_img_path}/*.png")):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = img.reshape(image_w,image_h,1)
    
    val_x.append(img/255.)
    val_y.append(1)

val_x = np.array(val_x)
val_y = np.array(val_y)

# Conver TF dataset

In [ ]:
train_x.shape, val_x.shape, train_y.shape, val_y.shape

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(buffer_size=100000).batch(64)
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(64)

# Def Model

In [ ]:
def VGG16():
    vgg16 = tf.keras.applications.vgg16.VGG16(include_top=False, weights=None, input_shape=input_shape)
    
    model = tf.keras.models.Sequential([
        vgg16,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

In [ ]:
def ResNet50(lr=0.001):
    resnet50 = tf.keras.applications.resnet50.ResNet50(include_top=False, weights=None, input_shape=input_shape)
    
    model = tf.keras.models.Sequential([
        resnet50,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

In [ ]:
def ResNet50V2(lr=0.001):
    resnet50v2 = tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, weights=None, input_shape=input_shape)
    
    model = tf.keras.models.Sequential([
        resnet50v2,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

In [ ]:
def ResNet101():
    resnet101 = tf.keras.applications.resnet.ResNet101(include_top=False, weights=None, input_shape=input_shape)
    
    model = tf.keras.models.Sequential([
        resnet101,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

In [ ]:
def InceptionV3():
    inceptionv3 = tf.keras.applications.inception_v3.InceptionV3(include_top=False, weights=None, input_shape=input_shape)
    
    model = tf.keras.models.Sequential([
        inceptionv3,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

In [ ]:
def EfficientNetB0():
    efficientnetb0 = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights=None, input_shape=input_shape)
    
    model = tf.keras.models.Sequential([
        efficientnetb0,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

In [ ]:
def EfficientNetV2B0(lr=0.001):
    efficientnetv2b0 = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False, weights=None, input_shape=input_shape)
    
    model = tf.keras.models.Sequential([
        efficientnetv2b0,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

In [ ]:
def EfficientNetV2B1(lr=0.001):
    efficientnetv2b1 = tf.keras.applications.efficientnet_v2.EfficientNetV2B1(include_top=False, weights=None, input_shape=input_shape)
    
    model = tf.keras.models.Sequential([
        efficientnetv2b1,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

In [ ]:
def DenseNet121():
    densenet121 = tf.keras.applications.densenet.DenseNet121(include_top=False, weights=None, input_shape=input_shape)
    
    model = tf.keras.models.Sequential([
        densenet121,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

In [ ]:
def DCAv1(lr=0.001):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(50, (5,5), 
                             activation='relu', padding='same',
                             input_shape=(256, 256, 1)))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))

    model.add(tf.keras.layers.Conv2D(70, (3,3), 
                             activation='relu', padding='same',
                             input_shape=(64, 64, 50)))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))

    model.add(tf.keras.layers.Conv2D(70, (3,3), 
                             activation='relu', padding='same',
                             input_shape=(32, 32, 70)))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

In [ ]:
def DCAv2():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(70, (5,5), 
                             activation='relu', padding='same',
                             input_shape=(256, 256, 1)))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))

    model.add(tf.keras.layers.Conv2D(70, (3,3), 
                             activation='relu', padding='same',
                             input_shape=(64, 64, 50)))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))

    model.add(tf.keras.layers.Conv2D(70, (3,3), 
                             activation='relu', padding='same',
                             input_shape=(32, 32, 70)))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy'])
    return model

# Train

## set folder

In [ ]:
folder_name = "ResNet50"
if not os.path.exists(f"./model"):
    os.mkdir(f"./model")
    print("./model")
if not os.path.exists(f"./img"):
    os.mkdir(f"./img")
    print("./img")
if not os.path.exists(f"./model/{folder_name}"):
    os.mkdir(f"./model/{folder_name}")
    print(f"create ./model/{folder_name}")
else:
    shutil.rmtree(f"./model/{folder_name}")
    os.mkdir(f"./model/{folder_name}")
    print(f"reset ./model/{folder_name}")
if not os.path.exists(f"./img/{folder_name}"):
    os.mkdir(f"./img/{folder_name}")
    print(f"create ./img/{folder_name}")

## lr schedules create

In [ ]:
lr_decayed_fn = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=30, alpha=0.001)

## model create

In [ ]:
model = ResNet50()
model.summary()

## callback create

In [ ]:
save_path = f"./model/{folder_name}/"

checkpoint_path = save_path+"/model_{epoch}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=0)

## model fit

In [ ]:
# history = model.fit(train_dataset, 
#                     epochs=30, 
#                     validation_data=val_dataset, 
#                     callbacks=[cp_callback])
history = model.fit(train_x, train_y,
                    epochs=30, 
                    validation_data=(val_x, val_y),
                    batch_size=64,
                    callbacks=[cp_callback])

## result

In [ ]:
idx = 14

plt.text(idx, 0.97, f'epoch = {idx+1}', ha='center', va='bottom', size = 10)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.text(idx, history.history['loss'][idx], '%.2f' %  history.history['loss'][idx], ha='center', va='bottom', size = 10)
plt.text(idx, history.history['val_loss'][idx], '%.2f' %  history.history['val_loss'][idx], ha='center', va='bottom', size = 10)

plt.axvline(idx, 0, 1, color='lightgray', linestyle='--', linewidth=2)

plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'])
plt.savefig(f"./img/{folder_name}/loss.png")
plt.show()

plt.text(idx, 0.97, f'epoch = {idx+1}', ha='center', va='bottom', size = 10)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.text(idx, history.history['accuracy'][idx], '%.2f' %  history.history['accuracy'][idx], ha='center', va='bottom', size = 10)
plt.text(idx, history.history['val_accuracy'][idx], '%.2f' %  history.history['val_accuracy'][idx], ha='center', va='bottom', size = 10)

plt.axvline(idx, 0, 1, color='lightgray', linestyle='--', linewidth=2)

plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy','val_accuracy'])
plt.savefig(f"./img/{folder_name}/acc.png")
plt.show()

# Model Load

In [ ]:
!unzip /content/drive/MyDrive/server_security/malware_model.zip -d ./malware_model

In [ ]:
epoch = 16
checkpoint_path = f"./malware_model/model_{epoch}.ckpt"

In [ ]:
malware_model = ResNet50()
malware_model.load_weights(checkpoint_path)

## Evaluate

In [ ]:
results = malware_model.evaluate(val_x, val_y)
print("test loss, test acc:", results)

## Predict

In [ ]:
prediction = malware_model.predict(val_x)
prediction

# Save Goole Drive

In [ ]:
!zip -r ResNet50.zip ./model/ResNet50/model_14.*

In [ ]:
!mkdir /content/drive/MyDrive/server_security/colab/KISA/Malware/img/ResNet50
!mv ./ResNet50.zip /content/drive/MyDrive/server_security/colab/KISA/Malware/model
!mv ./img/ResNet50/* /content/drive/MyDrive/server_security/colab/KISA/Malware/img/ResNet50